<img src="./img/logoconvexbrancomini2.png"  align="right"/>

<!--
<img src="./img/logoconvexbrancomini2.png"  align="right"/>
-->
# Resource Allocation Problem

<!--
<img src="./img/logoboxverde.png" align="right"/>
-->
__by [Daniel Cinalli](http://www.cinalli.com.br)__ - DSc Artificial Intelligence

## Uncapacitated Facility Location - Problem #04



<br/><br/> 
## Notes:

* Coded in Python 3.x
* Using [Anaconda](https://www.anaconda.com/) is recommended
* Run the notebook `online` at [binder](https://mybinder.org/v2/gh/drcinalli/Artificial-Intelligence-and-Transformation/master) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/drcinalli/Artificial-Intelligence-and-Transformation/master)
<!-- * [nbviewer](https://nbviewer.jupyter.org/) allows you to switch the notebooks "slides" mode-->

<br> </br>
### Table of Contents

- [Problem](#prob)
- [Simplex](#simplex)
- [Random Heuristic](#ramdom)


<br>
<br>


<a id='prob'></a>
## Problem #04

<br>
Facilities <br>
$|I| = 50$ 
<br>
<br>
Clients <br>
$|J| = 50$ 


<br> 
<br>


<a id='simplex'></a>
### Simplex (exact)



In [3]:
from itertools import product
from math import sqrt
import gurobipy as gp
from gurobipy import GRB

# Get Clients and Facilities
def getFacilities_Clients(file_list):
    return int(file_list[0]), int(file_list[1])

# Get Facilities Fixed Costs
def getFacilities_Capacity_FixedCosts(file_list, num_facilities):
    shift = 2
    capacity = []
    cost = []
    
    #loop to get all i location costs
    for i in range(0,num_facilities*2,2):
        capacity.append(int(file_list[i+shift]))
        cost.append(int(file_list[i+1+shift].replace(".","")))
    
    return capacity, cost

# Get Demand and Allocation Costs for j(customer) to each i(client)
def getClient_Demand_AllocationCosts(file_list, num_facilities, num_customers):
    shift = 2 + (num_facilities*2)
    demand = []
    allocation_cost = []
    
    #loop to get all j Clients 
    j=0
    for r in range(0,num_customers):
    
        #get demand
        demand.append(int(file_list[j+shift]))

        #loop to get all i location costs
        for i in range(0,num_facilities):
            allocation_cost.append(float(file_list[j+1+i+shift]))
            
        #fix j
        j += num_facilities+1
 
    
    return demand, allocation_cost

#Read File from OR datasets
fileName='datasets/ORcap133.txt'
ORlist = []

with open(fileName, "r") as f:
    ORlist = f.read().split()
    
##### Sets and Indices #####
num_facilities, num_customers = getFacilities_Clients(ORlist)
capacity, fixed_cost = getFacilities_Capacity_FixedCosts(ORlist, num_facilities)
cartesian_prod = list(product(range(num_customers), range(num_facilities)))
# shipping costs
demand, alloc_cost = getClient_Demand_AllocationCosts(ORlist, num_facilities, num_customers)
shipping_cost = dict(zip(cartesian_prod, alloc_cost))



# MIP  model formulation
m = gp.Model('UFLP')


##### Decision Variable #####
x = m.addVars(num_facilities, vtype=GRB.BINARY, name='x')
y = m.addVars(cartesian_prod, ub=1, vtype=GRB.CONTINUOUS, name='y')

##### Constraints #####
m.addConstrs((y[(c,f)] <= x[f] for c,f in cartesian_prod), name='Shipping')
m.addConstrs((gp.quicksum(y[(c,f)] for f in range(num_facilities)) == 1 for c in range(num_customers)), name='Demand')

##### Objective Function #####
m.setObjective(x.prod(fixed_cost)+y.prod(shipping_cost), GRB.MINIMIZE)

m.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 2550 rows, 2550 columns and 7500 nonzeros
Model fingerprint: 0x8adc286f
Variable types: 2500 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+02, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 3062847.1500
Presolve removed 2094 rows and 2069 columns
Presolve time: 0.02s
Presolved: 456 rows, 481 columns, 1258 nonzeros
Found heuristic solution: objective 1025990.0875
Variable types: 289 continuous, 192 integer (192 binary)

Root relaxation: objective 8.930767e+05, 159 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    893076.71250 893076.713  0.00%     -

In [5]:
# display optimal values of decision variables

for facility in x.keys():
    if (abs(x[facility].x) > 1e-6):
        print(f"\nBuild a warehouse at location {facility + 1}.")

# Shipments from facilities to customers.

for customer, facility in y.keys():
    if (abs(y[customer, facility].x) > 1e-6):
        print(f"\nClient {customer + 1} receives {round(100*y[customer, facility].x, 2)} % of its demand  from Warehouse {facility + 1} .")

#for v in m.getVars():
#    print(v.varname, v.x)

print(f"\nOptimal total:", m.objVal)

m.write('UFLP_04_Simplex.lp')


Build a warehouse at location 6.

Build a warehouse at location 23.

Build a warehouse at location 25.

Build a warehouse at location 27.

Build a warehouse at location 34.

Build a warehouse at location 45.

Build a warehouse at location 46.

Build a warehouse at location 49.

Client 1 receives 100.0 % of its demand  from Warehouse 25 .

Client 2 receives 100.0 % of its demand  from Warehouse 25 .

Client 3 receives 100.0 % of its demand  from Warehouse 6 .

Client 4 receives 100.0 % of its demand  from Warehouse 49 .

Client 5 receives 100.0 % of its demand  from Warehouse 25 .

Client 6 receives 100.0 % of its demand  from Warehouse 6 .

Client 7 receives 100.0 % of its demand  from Warehouse 34 .

Client 8 receives 100.0 % of its demand  from Warehouse 23 .

Client 9 receives 100.0 % of its demand  from Warehouse 25 .

Client 10 receives 100.0 % of its demand  from Warehouse 25 .

Client 11 receives 100.0 % of its demand  from Warehouse 23 .

Client 12 receives 100.0 % of its dema

<br>
<br>

<a id='xxx'></a>
### Heuristica



In [6]:
#very naive/simple
#for each client, choose randomly one of the Facilities available
import random

result=[]
#choose the Facility for each customer
for i in range(num_customers):
    result.append((i,random.randint(0, num_facilities-1)))
   

#remove duplication of facilities in order to print properly
facs=[]
for i in result:
    facs.append(i[1])
#print(result)
#print("xxx")
#print (facs)
facs=list(set(facs))
    
#print("xxx")
print (facs)

#calculate the setup_cost
totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

#print(totalC)

#calculate the shipping cost
for i in result: 
    totalC += shipping_cost.get(i)
    #print (shipping_cost.get(i))
    
print(totalC)



#setup_cost
#cost_per_mile*compute_distance(customers[c], facilities[f]) 
#list(set(output))

[0, 1, 3, 4, 6, 7, 8, 9, 14, 15, 18, 19, 22, 23, 24, 25, 27, 30, 31, 32, 34, 35, 39, 40, 41, 44, 46, 48, 49]
3357568.675


In [7]:
for i in facs:
    print(f"\nBuild a warehouse at location {i + 1}.")

for i in result:
    print(f"\nClient {i[0] + 1} receives 100% of its demand  from Warehouse {i[1] + 1} .")

print(f"\nOptimal total:", totalC)


Build a warehouse at location 1.

Build a warehouse at location 2.

Build a warehouse at location 4.

Build a warehouse at location 5.

Build a warehouse at location 7.

Build a warehouse at location 8.

Build a warehouse at location 9.

Build a warehouse at location 10.

Build a warehouse at location 15.

Build a warehouse at location 16.

Build a warehouse at location 19.

Build a warehouse at location 20.

Build a warehouse at location 23.

Build a warehouse at location 24.

Build a warehouse at location 25.

Build a warehouse at location 26.

Build a warehouse at location 28.

Build a warehouse at location 31.

Build a warehouse at location 32.

Build a warehouse at location 33.

Build a warehouse at location 35.

Build a warehouse at location 36.

Build a warehouse at location 40.

Build a warehouse at location 41.

Build a warehouse at location 42.

Build a warehouse at location 45.

Build a warehouse at location 47.

Build a warehouse at location 49.

Build a warehouse at locat

In [8]:
len(facs)

29